In [1]:
import os
import glob
import pandas as pd
import numpy as np



In [2]:
# Define the directory containing the results
results_dir = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Window/entropy_results/CausalImpact/"

# Load all participant files
file_paths = glob.glob(os.path.join(results_dir, "*.csv"))

# Read and concatenate all data
df_list = [pd.read_csv(file) for file in file_paths]
df = pd.concat(df_list, ignore_index=True)

# Convert time column to datetime
df['Gaze_Time'] = pd.to_datetime(df['Gaze_Time'])

# Sort by time
df = df.sort_values(by=['Participant_ID', 'Gaze_Time']).reset_index(drop=True)


In [3]:
from causalimpact import CausalImpact


# Define pre- and post-event windows
df['Time_Index'] = range(1, len(df) + 1)  # Assigns a numerical index to the time series

# Set up time series with entropy values
time_series = df[['Post_Entropy']].copy()  # Use post-event entropy for impact analysis
time_series.index = df['Time_Index']  # Use numerical time index

# Define pre-event (control) and post-event (treatment) periods
event_point = len(time_series) // 2  # Example: halfway through the dataset
pre_period = [1, event_point]  # Before event
post_period = [event_point + 1, len(time_series)]  # After event


In [4]:
print(time_series.head())
print(time_series.isna().sum())  # Check for NaNs
print("Pre-Period:", pre_period)
print("Post-Period:", post_period)
print("Min Time Index:", time_series.index.min())
print("Max Time Index:", time_series.index.max())


            Post_Entropy
Time_Index              
1               0.653000
2               0.932515
3               0.770499
4               0.427068
5               0.459757
Post_Entropy    0
dtype: int64
Pre-Period: [1, 2806]
Post-Period: [2807, 5613]
Min Time Index: 1
Max Time Index: 5613


In [5]:
print(time_series.shape)
time_series.head()

(5613, 1)


,Post_Entropy
Time_Index,
1,0.653000
2,0.932515
3,0.770499
4,0.427068
5,0.459757


In [6]:
print(time_series.loc[pre_period[0]:pre_period[1]].head())  # Pre-event period
print(time_series.loc[post_period[0]:post_period[1]].head())  # Post-event period


            Post_Entropy
Time_Index              
1               0.653000
2               0.932515
3               0.770499
4               0.427068
5               0.459757
            Post_Entropy
Time_Index              
2807            0.452935
2808            0.456639
2809            0.674997
2810            0.755798
2811            0.805846


In [9]:
print("NaN values:", time_series.isna().sum())
print("Inf values:", (time_series == np.inf).sum() + (time_series == -np.inf).sum())


NaN values: Post_Entropy    0
dtype: int64
Inf values: Post_Entropy    0
dtype: int64


In [15]:
import statsmodels.api as sm
print("Statsmodels imported successfully!")

Statsmodels imported successfully!


In [16]:
time_series = time_series.dropna()
time_series = time_series.replace([np.inf, -np.inf], np.nan).dropna()

ci = CausalImpact(time_series, pre_period, post_period, model_args={'niter': 1000, 'prior_level_sd': 0.1, 'pymc': False})
print(ci.summary())
ci.plot()



AttributeError: 'NoneType' object has no attribute 'loc'

In [17]:
print("Params:\n", ci.params)
print("Results:\n", ci.results if ci.results is not None else "No results generated")
print("Inferences:\n", ci.inferences if ci.inferences is not None else "No inferences generated")


Params:
 {'data':             Post_Entropy
Time_Index              
1               0.653000
2               0.932515
3               0.770499
4               0.427068
5               0.459757
...                  ...
5609            0.842905
5610            0.773489
5611            0.475900
5612            0.577328
5613            0.681094

[5613 rows x 1 columns], 'pre_period': [1, 49], 'post_period': [50, 100], 'model_args': {'niter': 1000, 'prior_level_sd': 0.1, 'pymc': False}, 'ucm_model': None, 'post_period_response': None, 'alpha': 0.05, 'estimation': 'MLE'}
Results:
 No results generated
Inferences:
 No inferences generated


In [18]:
ci = CausalImpact(data, pre_period, post_period, model_args={'pymc': False, 'prior_level_sd': 0.01})
print(ci.summary())
ci.plot()


AttributeError: 'NoneType' object has no attribute 'loc'

In [19]:
import statsmodels.api as sm

# Extract pre-period data
pre_data = data.iloc[pre_period[0]:pre_period[1]]

# Fit a simple linear model
X = sm.add_constant(range(len(pre_data)))
y = pre_data["y"]
model = sm.OLS(y, X).fit()

# Predict post-period
post_data = data.iloc[post_period[0]:post_period[1]]
X_post = sm.add_constant(range(len(post_data)))
predictions = model.predict(X_post)

print("Model coefficients:", model.params)
print("Predictions:\n", predictions.head())

# Plot results
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(pre_data.index, pre_data["y"], label="Pre-event Data")
plt.plot(post_data.index, post_data["y"], label="Post-event Data")
plt.plot(post_data.index, predictions, label="Predicted Data", linestyle="dashed")
plt.legend()
plt.show()


Model coefficients: const   -0.035398
x1      -0.007365
dtype: float64


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [20]:
print("Predictions:\n", pd.DataFrame(predictions).head())


Predictions:
           0
0 -0.035398
1 -0.042764
2 -0.050129
3 -0.057495
4 -0.064860


In [21]:
import logging

logging.basicConfig(level=logging.DEBUG)  # Enable debug logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

try:
    ci = CausalImpact(data, pre_period, post_period, model_args={'pymc': False})
    print("CausalImpact Model Parameters:", ci.params)
    print("CausalImpact Results:", ci.results if ci.results is not None else "No results generated")
    print("CausalImpact Inferences:", ci.inferences if ci.inferences is not None else "No inferences generated")
    print(ci.summary())
    ci.plot()
except Exception as e:
    print("CausalImpact error:", e)


CausalImpact Model Parameters: {'data':            y
0   0.496714
1  -0.138264
2   0.647689
3   1.523030
4  -0.234153
..       ...
95  1.536485
96  3.296120
97  3.261055
98  3.005113
99  2.765413

[100 rows x 1 columns], 'pre_period': [1, 49], 'post_period': [50, 100], 'model_args': {'pymc': False}, 'ucm_model': None, 'post_period_response': None, 'alpha': 0.05, 'estimation': 'MLE'}
CausalImpact Results: No results generated
CausalImpact Inferences: No inferences generated
CausalImpact error: 'NoneType' object has no attribute 'loc'


In [22]:
ci = CausalImpact(data, pre_period, post_period, model_args={'prior_level_sd': 0.01, 'pymc': False})
print(ci.summary())
ci.plot()


AttributeError: 'NoneType' object has no attribute 'loc'

In [23]:
import numpy as np
import pandas as pd
from causalimpact import CausalImpact

# Generate Fake Data
np.random.seed(42)
data = pd.DataFrame(np.random.randn(100, 1), columns=["y"])
data.iloc[50:, 0] += 3  # Introduce an effect

# Define Pre/Post Periods
pre_period = [1, 49]
post_period = [50, 100]

# Run CausalImpact
ci_test = CausalImpact(data, pre_period, post_period, model_args={'pymc': False})
print(ci_test.summary())
ci_test.plot()


AttributeError: 'NoneType' object has no attribute 'loc'

In [11]:
import numpy as np
import pandas as pd
from causalimpact import CausalImpact

# Generate fake data
np.random.seed(42)
data = pd.DataFrame(np.random.randn(100, 1), columns=["y"])
data.iloc[50:, 0] += 3  # Introduce an effect

# Define pre/post periods
pre_period = [1, 49]
post_period = [50, 100]

# Run CausalImpact
ci = CausalImpact(data, pre_period, post_period)
ci.plot()
print(ci.summary())


AttributeError: 'NoneType' object has no attribute 'iloc'

<Figure size 1500x1200 with 0 Axes>

In [13]:
ci = CausalImpact(data, pre_period, post_period, model_args={'pymc': False})
print(ci.summary())
ci.plot()


AttributeError: 'NoneType' object has no attribute 'loc'

In [14]:
import pymc as pm
print("PyMC imported successfully!")


PyMC imported successfully!


In [12]:
try:
    ci = CausalImpact(data, pre_period, post_period)
    print("CausalImpact ran successfully.")
    print(ci.summary())
except Exception as e:
    print("CausalImpact error:", e)


CausalImpact ran successfully.
CausalImpact error: 'NoneType' object has no attribute 'loc'


In [10]:
from causalimpact import CausalImpact
print("CausalImpact imported successfully!")

CausalImpact imported successfully!


In [7]:
print("Series Data:\n", ci.series.head())
print("Params:\n", ci.params)
print("Results:\n", ci.results if ci.results is not None else "No results generated")
print("Inferences:\n", ci.inferences if ci.inferences is not None else "No inferences generated")


NameError: name 'ci' is not defined

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing

# Fit a time series model to check if trend estimation works
pre_data = time_series.iloc[:2806]
post_data = time_series.iloc[2807:]

model = ExponentialSmoothing(pre_data, trend='add', seasonal=None, damped_trend=True)
fit = model.fit()

# Predict on post-period
forecast = fit.forecast(steps=len(post_data))

print("Model successfully fit. Forecast results:")
print(forecast.head())

# Plot results
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(pre_data, label="Pre-event Data")
plt.plot(post_data, label="Post-event Data")
plt.plot(forecast, label="Forecasted Data", linestyle="dashed")
plt.legend()
plt.show()


In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)  # Enable debug logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

try:
    ci = CausalImpact(time_series, pre_period, post_period, model_args={'pymc': False})
    print(ci.summary())
    ci.plot()
except Exception as e:
    print("CausalImpact Error:", e)


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

pre_data = time_series.iloc[:2806]
post_data = time_series.iloc[2807:]

# Fit an ARIMA model
model = ARIMA(pre_data, order=(1,1,1))
fit = model.fit()

# Forecast the post-event period
forecast = fit.forecast(steps=len(post_data))

# Plot results
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(pre_data, label="Pre-event Data")
plt.plot(post_data, label="Post-event Data")
plt.plot(forecast, label="ARIMA Forecast", linestyle="dashed")
plt.legend()
plt.show()


In [ ]:
ci = CausalImpact(time_series, pre_period, post_period, model_args={'niter': 1000, 'prior_level_sd': 0.01, 'pymc': False})
print(ci.summary())
ci.plot()


In [ ]:
print("Series Data:\n", ci.series.head())
print("Params:\n", ci.params)
print("Results:\n", ci.results if ci.results is not None else "No results generated")
print("Inferences:\n", ci.inferences if ci.inferences is not None else "No inferences generated")


In [ ]:
print("Pre-period shape:", time_series.loc[pre_period[0]:pre_period[1]].shape)
print("Post-period shape:", time_series.loc[post_period[0]:post_period[1]].shape)


In [ ]:
print("NaN values:", time_series.isna().sum())
print("Inf values:", (time_series == np.inf).sum() + (time_series == -np.inf).sum())


In [ ]:
print("Series Data:\n", ci.series.head())
print("Params:\n", ci.params)
print("Results:\n", ci.results if ci.results is not None else "No results generated")
print("Inferences:\n", ci.inferences if ci.inferences is not None else "No inferences generated")


In [ ]:
try:
    print(ci_test.summary())
except Exception as e:
    print("Error generating summary:", e)


In [ ]:
ci_test = CausalImpact(test_series, test_pre_period, test_post_period, model_args={'pymc': False})
print(ci_test.summary())
ci_test.plot()


In [ ]:
try:
    ci = CausalImpact(time_series, pre_period, post_period)
    print(ci.summary())
    ci.plot()
except Exception as e:
    print("Error running CausalImpact:", e)
